In [48]:
%run ../services/imports.py
%run ../services/function.py
%run ../services/classifier_evaluation.py

In [49]:
df_poscomp = pd.read_csv('../dados/df_poscompTest.csv')

In [4]:
X_dataset = df_poscomp.loc[:,['IDADE','SEXO','REGIAO','ESTADO','matematica', 
                               'fund_computacao','tec_computacao','total','linhas_pesquisas']]

In [5]:
# X_dataset['ESPECIALIDADE'] = filtered_df.ESPECIALIDADE
# X_dataset['ESPECIALIDADE'] = X_dataset['ESPECIALIDADE'].astype('category')

In [6]:
# colunas_a_padronizar = ['matematica', 'fund_computacao', 'tec_computacao']
# scaler = StandardScaler()
# X_dataset[colunas_a_padronizar] = scaler.fit_transform(X_dataset[colunas_a_padronizar])

In [7]:
def encode_categorical_columns(df, columns):
    le = LabelEncoder()
    for col in columns:
        df[col] = le.fit_transform(df[col])
    return df

categorical_columns = ['SEXO','REGIAO','ESTADO']
X_dataset = encode_categorical_columns(X_dataset, categorical_columns)


In [8]:
X = X_dataset.drop(['linhas_pesquisas'], axis=1)
y = X_dataset['linhas_pesquisas']

In [9]:
X.shape, y.shape

((8563, 8), (8563,))

In [10]:
y.value_counts()

linhas_pesquisas
inteligência artificial    3102
sistemas de computação     2266
engenharia de software     2077
redes de computadores      1118
Name: count, dtype: int64

### Dividir a base

In [46]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42,stratify=y)

In [47]:
y_train.value_counts()

linhas_pesquisas
inteligência artificial    2171
sistemas de computação     1586
engenharia de software     1454
redes de computadores       783
Name: count, dtype: int64

In [ ]:
print("Shape of x_train : ",x_train.shape)
print("Shape of x_test  : ",x_test.shape)
print("Shape of y_train : ",y_train.shape)
print("Shape of y_test  : ",y_test.shape)

In [44]:
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import EditedNearestNeighbours

smote = SMOTE(random_state=42)

smote_enn = SMOTEENN(random_state=42, smote=smote)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)

In [45]:
y_resampled.value_counts()

linhas_pesquisas
redes de computadores      1434
engenharia de software      660
sistemas de computação      603
inteligência artificial     232
Name: count, dtype: int64

In [13]:
x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42, stratify=y_resampled)

In [14]:
parametros = {
    'n_estimators': [300,500,1000],  # Você pode ajustar o número de árvores conforme necessário
    'criterion': ['entropy'],
    'max_depth': [4,5,6,7,8],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4],
    # 'min_weight_fraction_leaf': [0.0, 0.1, 0.2],
    'max_features': [5,6,7,8,9,10],
    # 'max_leaf_nodes': [None, 10, 20, 30],
    # 'min_impurity_decrease': [0.0, 0.1, 0.2],
    # 'bootstrap': [True, False],
    # 'oob_score': [True, False],
    # 'n_jobs': [-1],  # Use -1 para usar todos os processadores disponíveis
    # 'random_state': [None, 42],  # Use um valor fixo para repetibilidade
    # 'verbose': [0, 1, 2],
    # 'warm_start': [True, False],
    # 'class_weight': ['balanced', 'balanced_subsample'],
    # 'ccp_alpha': [0.0, 0.1, 0.2],
    # 'max_samples': [None, 0.7, 0.8, 0.9],
}

In [16]:
forest_clf = ClassifierWrapper(RandomForestClassifier(random_state=42), parametros)
forest_clf.grid_search(x_train, y_train, cv=3)
forest_clf.train_and_print_scores(x_train, y_train, x_test, y_test)

Fitting 3 folds for each of 540 candidates, totalling 1620 fits
Best Parameters:
{'criterion': 'entropy', 'max_depth': 8, 'max_features': 6, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best Score:
0.7541547866738588
Best Estimator:
RandomForestClassifier(criterion='entropy', max_depth=8, max_features=6,
                       n_estimators=300, random_state=42)
Train Result:
Accuracy Score: 86.20%
__________________________________________________________________________
CLASSIFICATION REPORT:
           engenharia de software  inteligência artificial  \
precision                0.878307                 0.936508   
recall                   0.718615                 0.728395   
f1-score                 0.790476                 0.819444   
support                462.000000               162.000000   

           redes de computadores  sistemas de computação  accuracy  \
precision               0.820168                0.957865  0.861951   
recall                  0.9

In [ ]:
parametros = {
    'C': [3,4,5],
    # 'degree': [1,2,3],
    'gamma': [0.1,0.2,0.5],
    'coef0': [0.1],
    'shrinking': [True,False],
    # 'probability': [True, False],
    # 'tol': [1e-4],
    # 'cache_size': np.arange(1,11,2),
    'class_weight': ['balanced'],
    # 'verbose': [True],
    # 'max_iter': [100, 1000, -1],  # -1 indica nenhum limite
    # 'decision_function_shape': ['ovr'],
    # 'break_ties': [True],
} 

In [ ]:
svm_clf = ClassifierWrapper(SVC(random_state=42, kernel='rbf'), parametros)
svm_clf.grid_search(x_train, y_train, cv=3)
svm_clf.train_and_print_scores(x_train, y_train, x_test, y_test)

In [ ]:
svm_clf = ClassifierWrapper(SVC(random_state=42, kernel='poly'), parametros)
svm_clf.grid_search(x_train, y_train, cv=2)
svm_clf.train_and_print_scores(x_train, y_train, x_test, y_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

algorithms = [
    ('Árvore de Decisão (C4.5)', DecisionTreeClassifier(random_state=42)),
    ('Random Forest', RandomForestClassifier(random_state=42)),
    ('SVM', SVC(random_state=42, kernel='poly')),
    ('Rede Neural', MLPClassifier(random_state=42, hidden_layer_sizes=(100,100))),
]

for name, classifier in algorithms:
    classifier.fit(x_train, y_train)
    train_predictions = classifier.predict(x_train)
    test_predictions = classifier.predict(x_test)
    
    train_accuracy = accuracy_score(y_train, train_predictions)
    test_accuracy = accuracy_score(y_test, test_predictions)
    
    precision = precision_score(y_test, test_predictions, average='macro')
    recall = recall_score(y_test, test_predictions, average='macro')
    f1 = f1_score(y_test, test_predictions, average='macro')
    
    conf_matrix = confusion_matrix(y_test, test_predictions)
    
    print(f'{name} - Acurácia no Conjunto de Treinamento: {train_accuracy * 100:.2f}%')
    print(f'{name} - Acurácia no Conjunto de Teste: {test_accuracy * 100:.2f}%')
    print(f'{name} - Precisão: {precision * 100:.2f}%')
    print(f'{name} - Recall: {recall * 100:.2f}%')
    print(f'{name} - F1-Score: {f1 * 100:.2f}%')
    print(f'{name} - Matriz de Confusão:\n{conf_matrix}\n')
    

In [ ]:
classifier = RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=10,
                       n_estimators=200, n_jobs=-1, random_state=42,
                       warm_start=True)
classifier.fit(x_train, y_train)
train_predictions = classifier.predict(x_train)
test_predictions = classifier.predict(x_test)

In [ ]:
classifier.predict([[21.0,  1.0,  4.0, 22.0, 9.0, 16.0, 8.0, 33.0]])